[NWB (Neurodata Without Borders)](https://www.nwb.org/) is a data format proposed to become a data standard for neurophysiology. In this blog post, I'll try to use it and demonstorate how to load the data and to work on the data.

# Download the data

Here, I try to download the data used in the paper titled [Neurons detect cognitive boundaries to structure episodic memories in humans](https://www.nature.com/articles/s41593-022-01020-w). I didn't have the strong reason to choose it... I just thought human data is more familiar to me than some other animal data. In this paper, the authors say:

> We recorded single-neuron activity from individuals with drug-resistant epilepsy implanted with depth electrodes while testing their memory for the content of video clips with two kinds of embedded cognitive boundaries: soft boundaries (SBs) and hard boundaries (HBs). SBs are episodic transitions between related events within the same movie, while HBs are episodic transitions between two unrelated movies.

So their participants are already implanted with electrodes, and they measured participant's neural activity during watching some videos they provided. The group of video clips have two categories: soft and hard boundary. The video set with soft boundary has video clips only from the same movie, and the video set with hard boundary has video clips from several random movies. I don't know how these setups matter, but let's see the data. We can find lots of open NWB data (319 now) on [dandi archive](https://dandiarchive.org/). We can download dandi data from the [web page](https://dandiarchive.org/dandiset/000207/) directly or using the [CLI](https://dandi.readthedocs.io/en/latest/) that you can download from PyPI. Here I use the Python API packed with the CLU instead, because this post is entirely written on a Jupyter notebook...

In [16]:
from pathlib import Path

import numpy as np
import pynwb
from dandi.download import download as dandi_download
from matplotlib import pyplot as plt
from nwbwidgets import nwb2widget

DATA_DIR = Path("../data/nwb")

Let's download data.

In [2]:
if not DATA_DIR.exists():
    DATA_DIR.mkdir(parents=True)
    dandi_download("DANDI:000207/0.220216.0323", output_dir=DATA_DIR)

PATH                                          SIZE     DONE            DONE% CHECKSUM STATUS          MESSAGE   
000207/dandiset.yaml                                                                  done            updated   
000207/sub-16/sub-16_ses-16_ecephys+image.nwb 2.7 MB   2.7 MB           100%    ok    done                      
000207/sub-11/sub-11_ses-11_ecephys+image.nwb 1.4 MB   1.4 MB           100%    ok    done                      
000207/sub-1/sub-1_ses-1_ecephys+image.nwb    5.2 MB   5.2 MB           100%    ok    done                      
000207/sub-10/sub-10_ses-10_ecephys+image.nwb 1.5 MB   1.5 MB           100%    ok    done                      
000207/sub-18/sub-18_ses-18_ecephys+image.nwb 3.5 MB   3.5 MB           100%    ok    done                      
000207/sub-13/sub-13_ses-13_ecephys+image.nwb 2.2 MB   2.2 MB           100%    ok    done                      
000207/sub-14/sub-14_ses-14_ecephys+image.nwb 2.9 MB   2.9 MB           100%    ok    done      

It looks like that the data is downloaded under `000207` directory in the data directory I specified, and there are 19 subdirectories for each participant.
Let's prepare a convenient function for reading the data.

In [3]:
def load_nwb(session_id: int) -> pynwb.NWBFile:
    """Load an NWB file. session_id should be an integer from 1 to 19."""
    path = DATA_DIR.joinpath(
        f"000207/sub-{session_id}/sub-{session_id}_ses-{session_id}_ecephys+image.nwb"
    )
    return pynwb.NWBHDF5IO(path, mode="r").read()

While I already roughly read the paper, I have no idea what these files contain. 
To help me check the data content, NWB provides a separated package called [nwbwidgets](https://nwb-overview.readthedocs.io/en/latest/tools/nwbwidgets/nwbwidgets.html) that creates an interactive [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/stable/).
Let's load a data and visualize it.

In [4]:
%matplotlib widget
nwb10 = load_nwb(10)
nwb2widget(nwb10)

/home/yuji/Documents/rlog2/.venv/lib/python3.11/site-packages/pynwb/core.py:47: UserWarning: OpticalSeries 'StimulusPresentation_encoding': The number of frame indices in 'starting_frame' should have the same length as 'external_file'.
  warn(error_msg)
/home/yuji/Documents/rlog2/.venv/lib/python3.11/site-packages/pynwb/core.py:47: UserWarning: OpticalSeries 'StimulusPresentation_encoding': Either external_file or data must be specified (not None), but not both.
  warn(error_msg)


It didn't helped me understand the data much..., but at least, I now know that `units/Session Raster` shows me a raster plot of the spiking data. So, how can I analyze these data? And, how can I get the raw array of spikes? Let's just print out the `units`.

In [6]:
nwb10.units

units pynwb.misc.Units at 0x140382092456016
Fields:
  colnames: ['spike_times' 'electrodes']
  columns: (
    spike_times_index <class 'hdmf.common.table.VectorIndex'>,
    spike_times <class 'hdmf.common.table.VectorData'>,
    electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  )
  description: units table
  id: id <class 'hdmf.common.table.ElementIdentifiers'>
  waveform_unit: volts

I found that it is an instance of [`Units`](https://pynwb.readthedocs.io/en/stable/pynwb.misc.html) class, which represents event times of observed units. It provides us a convenient method `get_unit_spike_times(unit: int | Sequence[int])`, which returns a `NumPy` array represents the spike timing of given a unit of units.

In [11]:
nwb10.units.get_unit_spike_times(0)

array([6.57545000e-02, 2.23144750e-01, 6.55594500e-01, ...,
       2.88324675e+03, 2.88364036e+03, 2.88391571e+03])

Or, we can just access contents by `units["spike_times"][unit]`. 

# Visualize the spikes

I found a nice example of plotting it in the [pynwb document](https://pynwb.readthedocs.io/en/stable/tutorials/general/read_basics.html#visualize-spiking-activity-relative-to-stimulus-onset)

In [15]:
before = 1.0  # in seconds
after = 3.0

def plot_spikes(units: list[int]) -> None:
    for unit in range(3):
        unit_spike_times = nwbfile.units["spike_times"][unit]
        trial_spikes = []
        for time in stim_on_times:
            # Compute spike times relative to stimulus onset
            aligned_spikes = unit_spike_times - time
            # Keep only spike times in a given time window around the stimulus onset
            aligned_spikes = aligned_spikes[
                (-before < aligned_spikes) & (aligned_spikes < after)
            ]
            trial_spikes.append(aligned_spikes)
        fig, axs = plt.subplots(2, 1, sharex="all")
        plt.xlabel("time (s)")
        axs[0].eventplot(trial_spikes)

        axs[0].set_ylabel("trial")
        axs[0].set_title("unit {}".format(unit))
        axs[0].axvline(0, color=[0.5, 0.5, 0.5])

        axs[1].hist(np.hstack(trial_spikes), 30)

(27,)

In [18]:
nwb10.units["spike_times"][0]

array([6.57545000e-02, 2.23144750e-01, 6.55594500e-01, ...,
       2.88324675e+03, 2.88364036e+03, 2.88391571e+03])